In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Input, Dense, Reshape, Flatten, LeakyReLU, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

print(sys.executable)
print(sys.version)


C:\Users\KIIT\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]


In [2]:
# Sample text
text = "Soumyadeep is also known as SD1 and Suryadipta is also known as SD2. " \
         "Soumyadeep is a AI engineer and Suryadipta is a Phramacist. " 

In [3]:
# Step 1: Tokenize at word level
tokenizer = Tokenizer()
print("Original text:", text)
print("Tokenizing text at word level...")
# Fit the tokenizer on the text
tokenizer.fit_on_texts([text])
print("Tokenized text:", tokenizer.texts_to_sequences([text]))
word_to_index = tokenizer.word_index
print("Word to index mapping:", word_to_index)
index_to_word = {v: k for k, v in word_to_index.items()}
print("Index to word mapping:", index_to_word)
total_words = len(index_to_word) + 1  # +1 for padding/indexing from 1
print("Total words:", total_words)

Original text: Soumyadeep is also known as SD1 and Suryadipta is also known as SD2. Soumyadeep is a AI engineer and Suryadipta is a Phramacist. 
Tokenizing text at word level...
Tokenized text: [[2, 1, 3, 4, 5, 9, 6, 7, 1, 3, 4, 5, 10, 2, 1, 8, 11, 12, 6, 7, 1, 8, 13]]
Word to index mapping: {'is': 1, 'soumyadeep': 2, 'also': 3, 'known': 4, 'as': 5, 'and': 6, 'suryadipta': 7, 'a': 8, 'sd1': 9, 'sd2': 10, 'ai': 11, 'engineer': 12, 'phramacist': 13}
Index to word mapping: {1: 'is', 2: 'soumyadeep', 3: 'also', 4: 'known', 5: 'as', 6: 'and', 7: 'suryadipta', 8: 'a', 9: 'sd1', 10: 'sd2', 11: 'ai', 12: 'engineer', 13: 'phramacist'}
Total words: 14


In [4]:
# Step 2: Create sequences of words
words = text.split()
seq_length = 3
sequences = []
labels = []

import string

for i in range(len(words) - seq_length):
    seq = words[i:i + seq_length]
    label = words[i + seq_length]
    # Remove punctuation and lowercase for each word
    seq_clean = [word.lower().strip(string.punctuation) for word in seq]
    print("Cleaned sequence:", seq_clean)
    label_clean = label.lower().strip(string.punctuation)
    print("Cleaned label:", label_clean)
    sequences.append([word_to_index[word] for word in seq_clean])
    labels.append(word_to_index[label_clean])

print("Sequences:", sequences)
print("Labels:", labels)

X = np.array(sequences)
print("Input sequences (X):", X)
print("Input shape:", X.shape)
y = to_categorical(labels, num_classes=total_words)
print("Output labels (y):", y)
print("Output shape:", y.shape)

Cleaned sequence: ['soumyadeep', 'is', 'also']
Cleaned label: known
Cleaned sequence: ['is', 'also', 'known']
Cleaned label: as
Cleaned sequence: ['also', 'known', 'as']
Cleaned label: sd1
Cleaned sequence: ['known', 'as', 'sd1']
Cleaned label: and
Cleaned sequence: ['as', 'sd1', 'and']
Cleaned label: suryadipta
Cleaned sequence: ['sd1', 'and', 'suryadipta']
Cleaned label: is
Cleaned sequence: ['and', 'suryadipta', 'is']
Cleaned label: also
Cleaned sequence: ['suryadipta', 'is', 'also']
Cleaned label: known
Cleaned sequence: ['is', 'also', 'known']
Cleaned label: as
Cleaned sequence: ['also', 'known', 'as']
Cleaned label: sd2
Cleaned sequence: ['known', 'as', 'sd2']
Cleaned label: soumyadeep
Cleaned sequence: ['as', 'sd2', 'soumyadeep']
Cleaned label: is
Cleaned sequence: ['sd2', 'soumyadeep', 'is']
Cleaned label: a
Cleaned sequence: ['soumyadeep', 'is', 'a']
Cleaned label: ai
Cleaned sequence: ['is', 'a', 'ai']
Cleaned label: engineer
Cleaned sequence: ['a', 'ai', 'engineer']
Cleaned 

In [5]:
# Step 3: One-hot encode inputs manually
X_one_hot = tf.keras.utils.to_categorical(X, num_classes=total_words) 
# y_one_hot = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [17]:
# Step 4: Build RNN model
model = Sequential([
    SimpleRNN(64, input_shape=(seq_length, total_words)),
    Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Step 5: Train model
model.fit(X_one_hot, y, epochs=300, verbose=0)
model.summary()        
model.weights      
model.variables       

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         5,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,900 (69.93 KB)

 Trainable params: 5,966 (23.30 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 11,934 (46.62 KB)

[<Variable path=sequential/simple_rnn/simple_rnn_cell/kernel, shape=(14, 64), dtype=float32, value=[[-1.56767547e-01 -1.25853390e-01  1.11443013e-01 -5.99087030e-02
    8.41424167e-02  2.50857502e-01  1.46742612e-01 -4.03270423e-02
    1.01147354e-01 -7.59938359e-02 -2.27487773e-01  1.18236840e-02
   -2.70840853e-01  5.32418787e-02  1.00699097e-01 -1.28158793e-01
    1.77183926e-01 -1.26214445e-01  2.67471105e-01  1.10228807e-01
   -2.64946967e-01  3.09546888e-02  9.91130173e-02  2.44790405e-01
   -1.71832174e-01 -2.63348252e-01 -2.38376975e-01  5.65944314e-02
   -1.01654664e-01  2.54630476e-01 -1.26680091e-01 -1.79534271e-01
    1.48805648e-01 -2.06164241e-01  4.90005910e-02  9.15628672e-02
   -2.20757395e-01  1.70810491e-01  6.71788454e-02  2.61936814e-01
   -2.22023815e-01 -1.04579598e-01  1.22158110e-01  1.76483124e-01
    2.20685601e-02 -1.49661660e-01 -9.33157951e-02  1.94629252e-01
    1.26807511e-01  1.87191129e-01  1.49316877e-01  1.86212927e-01
    1.83074832e-01  1.77869439e

In [8]:
from googletrans import Translator

# Step 6: Generate text
seed_text = "soumyadeep is also"  # Use only words from your vocabulary
generated_text = seed_text

for _ in range(10):
    seed_seq = [word_to_index[word] for word in generated_text.split()[-seq_length:]] # get last seq_length words
    x = np.array([seed_seq]) # reshape to match input shape
    x = tf.keras.utils.to_categorical(x, num_classes=total_words) # one-hot encode the input
    prediction = model.predict(x, verbose=0) # predict next word
    next_index = np.argmax(prediction) # get index of the predicted word
    next_word = index_to_word[next_index] # convert index to word
    generated_text += ' ' + next_word # append the predicted word to the generated text

print("The original text was:", text)
print("Generated text:", generated_text.capitalize())
translator = Translator()
translated = translator.translate(generated_text, src='en', dest='bengali')
print("Bengali Translation:")
print("Translated text", translated.text.capitalize())


The original text was: Soumyadeep is also known as SD1 and Suryadipta is also known as SD2. Soumyadeep is a AI engineer and Suryadipta is a Phramacist. 
Generated text: Soumyadeep is also known as sd2 soumyadeep is a ai engineer and suryadipta
Bengali Translation:
Translated text সৌমিয়াদীপ এসডি 2 হিসাবেও পরিচিত


LSTM MODEL BUILDING

In [9]:
# Step 1: Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(seq_length, total_words)),
    Dense(total_words, activation='softmax')
])
model.compile(loss='bce', optimizer='RMSprop', metrics=['accuracy'])

In [16]:
# Step 2: Train model
model.fit(X_one_hot, y, epochs=300, verbose=0)

# Summary
model.summary()
model.weights
model.variables

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        15,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,812 (190.68 KB)

 Trainable params: 16,270 (63.55 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 32,542 (127.12 KB)

[<Variable path=sequential_2/gru/gru_cell/kernel, shape=(14, 192), dtype=float32, value=[[-0.00811084 -0.11289869  0.01493002 ... -0.1147307   0.12588376
    0.14339301]
  [-0.21786587  0.04951379  0.08799435 ... -0.0254457   0.2729946
    0.08277229]
  [ 0.08267836 -0.20167445 -0.38896325 ...  0.1046454  -0.16142972
    0.23903303]
  ...
  [ 0.07640111  0.0230454  -0.37569645 ... -0.29164082  0.0784619
   -0.06657187]
  [-0.13268729 -0.17227603  0.06947731 ... -0.20813346 -0.2711072
    0.17693493]
  [-0.12707548  0.0854761   0.08670083 ...  0.0250425  -0.01971918
   -0.0150995 ]]>,
 <Variable path=sequential_2/gru/gru_cell/recurrent_kernel, shape=(64, 192), dtype=float32, value=[[-0.1525892  -0.02760535 -0.18259363 ... -0.10728732  0.17467298
    0.03029564]
  [-0.1825649   0.20743902  0.39813247 ... -0.15200649  0.10314722
   -0.20668536]
  [-0.07960753  0.19522978  0.33493102 ...  0.17241555 -0.28841728
   -0.16430047]
  ...
  [ 0.04996151  0.11753797  0.30855247 ...  0.17747487 -0

In [11]:
# Step 3: Generate text
seed_text = "soumyadeep is also"
generated_text = seed_text

for _ in range(10):
    seed_seq = [word_to_index[word] for word in generated_text.split()[-seq_length:]]
    x = np.array([seed_seq])
    x = tf.keras.utils.to_categorical(x, num_classes=total_words)
    prediction = model.predict(x, verbose=0)
    next_index = np.argmax(prediction)
    next_word = index_to_word[next_index]
    generated_text += ' ' + next_word

print("\nThe original text was:\n", text)
print("\nGenerated text:\n", generated_text.capitalize())


The original text was:
 Soumyadeep is also known as SD1 and Suryadipta is also known as SD2. Soumyadeep is a AI engineer and Suryadipta is a Phramacist. 

Generated text:
 Soumyadeep is also known as as and is is a known as and


GRU MODEL

In [12]:
# Step 1: Build GRU model
model = Sequential([
    GRU(64, input_shape=(seq_length, total_words)),
    Dense(total_words, activation='softmax')
])
model.compile(loss="bce", optimizer='Adam', metrics=['accuracy', 'mae'])


In [13]:
# Step 2: Train model
model.fit(X_one_hot, y, epochs=300, verbose=0)

In [14]:
# Step 3: Generate text
seed_text = "soumyadeep is also"
generated_text = seed_text

for _ in range(10):
    seed_seq = [word_to_index[word] for word in generated_text.split()[-seq_length:]]
    x = np.array([seed_seq])
    x = tf.keras.utils.to_categorical(x, num_classes=total_words)
    prediction = model.predict(x, verbose=0)
    next_index = np.argmax(prediction)
    next_word = index_to_word[next_index]
    generated_text += ' ' + next_word

print("\nThe original text was:\n", text)
print("\nGenerated text:\n", generated_text.capitalize())


The original text was:
 Soumyadeep is also known as SD1 and Suryadipta is also known as SD2. Soumyadeep is a AI engineer and Suryadipta is a Phramacist. 

Generated text:
 Soumyadeep is also known as sd2 soumyadeep is a ai engineer and suryadipta
